In [5]:
import pybamm
class Battery:
    def __init__(self, capacity, initial_soc=1, min_soc=0, charge_rate=100):
        self.capacity = capacity
        assert 0 <= initial_soc <= 1
        self.charge_level = capacity * initial_soc
        self._soc = initial_soc
        assert 0 <= min_soc <= self._soc
        self.min_soc = min_soc
        self.charge_rate = charge_rate

    @property
    def soc(self):
        return self.charge_level / self.capacity

    def discharge(self, power_amount, time_step):
        self.charge_level -= power_amount * time_step
        if self.charge_level < self.min_soc * self.capacity:
            self.charge_level = self.min_soc * self.capacity
        return power_amount * time_step

    def charge(self, power_amount, time_step):
        self.charge_level += power_amount * time_step
        if self.charge_level > self.capacity:
            self.charge_level = self.capacity
        return power_amount * time_step

In [6]:
param = pybamm.ParameterValues("Marquis2019")
param

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.4,
 'Cell cooling surface area [m2]': 0.0569,
 'Cell volume [m3]': 7.8e-06,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 0.680616,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Edge heat transfer coefficient [W.m-2.K-1]': 0.3,
 'Electrode height [m]': 0.137,
 'Electrode width [m]': 0.207,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Capiglia1999 at 0x28f2d3af0>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Capiglia1999 at 0x28f2d3a60>,
 'Electron charge [C]': 1.602176634e-19,
 'Faraday constant [C.mol-1]': 96485.33212,
 'Ideal gas constant [J.K-1.mol-1]': 8.314462618,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 19986.609595075,
 'Initial 

In [ ]:
model = pybamm.lithium_ion.DFN()
model.name


In [ ]:
sim = pybamm.Simulation(model)

In [ ]:
sim.solve([0, 3600])

In [ ]:
# sim.plot()
model.variable_names()

In [ ]:
models = [
    pybamm.lithium_ion.SPM(),
    pybamm.lithium_ion.SPMe(),
    pybamm.lithium_ion.DFN(),
]

In [ ]:
sims = []
for model in models:
    sim = pybamm.Simulation(model)
    sim.solve([0, 3600])
    sims.append(sim)

In [ ]:
pybamm.dynamic_plot(sims)


In [ ]:
model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model)
sim.solve([0, 3600])
model.variable_names()

In [ ]:
parameter_values = pybamm.ParameterValues("Chen2020")

In [ ]:
parameter_values

In [ ]:
import numpy as np


def my_current(t):
    return pybamm.sin(2 * np.pi * t / 60)


parameter_values["Current function [A]"] = my_current

In [ ]:
sim = pybamm.Simulation(model, parameter_values=parameter_values)
t_eval = np.arange(0, 121, 1)
sim.solve(t_eval=t_eval)
sim.plot(["Current [A]", "Voltage [V]"])

In [ ]:
experiment = pybamm.Experiment(
    [
        (
            "Discharge at C/10 for 10 hours or until 3.3 V",
            "Rest for 1 hour",
            "Charge at 1 A until 4.1 V",
            "Hold at 4.1 V until 50 mA",
            "Rest for 1 hour",
        )
    ]
    * 3
    + [
        "Discharge at 1C until 3.3 V",
    ]
)

model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, experiment=experiment)
sim.solve()
sim.plot()

In [ ]:
model = pybamm.lithium_ion.SPMe()
sim = pybamm.Simulation(model)
sim.solve([0, 3600])


In [ ]:
solution = sim.solution
solution = sim.solve([0, 3600])

t = solution["Time [s]"]
V = solution["Voltage [V]"]

V.entries

In [ ]:
t.entries

In [ ]:
import pybamm

# Load a standard parameter set directly
param = pybamm.ParameterValues("Marquis2019")

# Set up the battery model
model = pybamm.lithium_ion.DFN()

# Define the power values for discharge and charge
discharge_power_watts = 1  # 1 W discharge
charge_power_watts = -0.5  # -0.5 W charge (negative for charging)

# Create a power-controlled step for discharge and charge
discharge_step = pybamm.step.power(discharge_power_watts, duration=0.5*3600)  # 0.5 hours in seconds
charge_step = pybamm.step.power(charge_power_watts, duration=0.5*3600)  # 0.5 hours in seconds

# Create an experiment with these steps
experiment = pybamm.Experiment(
    [
        discharge_step,  # Discharge for 0.5 hours at 1 W
        charge_step,     # Charge for 0.5 hours at 0.5 W
        "Rest for 1 hour"  # Rest for 1 hour
    ]
)

# Create and run the simulation
sim = pybamm.Simulation(model, experiment=experiment, parameter_values=param)
sim.solve()
sim.plot()


In [ ]:
import pybamm

# Load a standard parameter set directly
param = pybamm.ParameterValues("Marquis2019")

# Set up the battery model
model = pybamm.lithium_ion.DFN()

# Define the power values for discharge and charge
discharge_power_watts = 1  # 1 W discharge
charge_power_watts = -0.5  # -0.5 W charge (negative for charging)

# Create a power-controlled step for discharge and charge
discharge_step = pybamm.step.power(discharge_power_watts, duration=0.5*3600)  # 0.5 hours in seconds
charge_step = pybamm.step.power(charge_power_watts, duration=0.5*3600)  # 0.5 hours in seconds

# Create an experiment with these steps
experiment = pybamm.Experiment(
    [
        discharge_step,  # Discharge for 0.5 hours at 1 W
        charge_step,     # Charge for 0.5 hours at 0.5 W
        "Rest for 1 hour"  # Rest for 1 hour
    ]
)

# Specify the desired output variables
output_variables = [
    # "Nominal cell capacity [A.h]",  # Total nominal capacity of the cell
    "Discharge capacity [A.h]",      # Total capacity discharged in Ah
    "Current [A]",
    "Voltage [V]",
]

# Create and run the simulation
sim = pybamm.Simulation(
    model,
    experiment=experiment,
    parameter_values=param,
    output_variables=output_variables
)
solution = sim.solve()

# Extract and store the nominal cell capacity
capacity_ah = 12000
model.options["calculate discharge energy"] = "true"
parameter_values = pybamm.ParameterValues("Ecker2015")
capacity = parameter_values["Nominal cell capacity [A.h]"]
print(capacity)
parameter_values.update({"Nominal cell capacity [A.h]": capacity_ah})
nominal_capacity = solution["Nominal cell capacity [A.h]"].data
print(f"Nominal cell capacity: {nominal_capacity[0]} A.h")

# Plot the results
sim.plot()


In [ ]:
class Battery:
    def __init__(self, capacity, initial_soc=1, min_soc=0, charge_rate=100):
        self.capacity = capacity
        assert 0 <= initial_soc <= 1
        self.charge_level = capacity * initial_soc
        self._soc = initial_soc
        assert 0 <= min_soc <= self._soc
        self.min_soc = min_soc
        self.charge_rate = charge_rate

    @property
    def soc(self):
        return self.charge_level / self.capacity

    def discharge(self, power_amount, time_step):
        self.charge_level -= power_amount * time_step
        if self.charge_level < self.min_soc * self.capacity:
            self.charge_level = self.min_soc * self.capacity
        return power_amount * time_step

    def charge(self, power_amount, time_step):
        self.charge_level += power_amount * time_step
        if self.charge_level > self.capacity:
            self.charge_level = self.capacity
        return power_amount * time_step

import pybamm

class PyBammBattery(Battery):
    def __init__(self, capacity, initial_soc=1, min_soc=0, charge_rate=100):
        super().__init__(capacity, initial_soc, min_soc, charge_rate)
        self.nominal_voltage = 120  # Nominal voltage in V
        self.capacity_wh = capacity  # Capacity in Watt-hours
        self.capacity_ah = self.capacity_wh / self.nominal_voltage  # Convert capacity from W.h to A.h
        self.model = pybamm.lithium_ion.DFN()
        self.model.options["calculate discharge energy"] = "true"
        parameter_values = self.model.default_parameter_values
        parameter_values.update({"Nominal cell capacity [A.h]": self.capacity_ah})
        self.simulation = pybamm.Simulation(self.model, parameter_values=parameter_values)
        self.simulation.set_initial_soc(initial_soc)

        # Print all available variables
        # print("Available variables in the model:")
        # for var in self.simulation.model.variables.keys():
        #     print(var)


    def charge(self, power_amount, duration):
        """
        Charge the battery with a given power for a specific duration.
        
        Args:
            power_amount (float): Power in Watts.
            duration (float): Duration in seconds.
        """
        # Negative power for charging
        charge_step = pybamm.step.power(-power_amount, duration=duration)
        
        # Create an experiment with this step
        experiment = pybamm.Experiment(["Discharge at 1 W for 1 hours"])
        
        # Run the simulation with the experiment
        self.simulation = pybamm.Simulation(self.model, experiment=experiment, parameter_values=self.simulation.parameter_values)
        solution = self.simulation.solve()

        # Print the final state of charge (SOC)
        # soc = solution["Discharge Capacity [A.h]"].data[-1]
        discharge_power_watts = solution["Discharge capacity [A.h]"].data[-1]
        print(f"discharge power watts: {discharge_power_watts:.2f}")
        discharge_energy = solution["Discharge energy [W.h]"].data[-1]
        print(f"discharge energy: {discharge_energy:.2f}")

        return solution
    
battery = PyBammBattery(capacity=5000)  # 5000 W.h capacity
output = battery.charge(power_amount=100, duration=360)  # Discharge at 100 W for 1 hour
# battery.charge(power_amount=50, duration=3600)  # Charge at 50 W for 1 hour
output = battery.charge(power_amount=100, duration=360)  # Discharge at 100 W for 1 hour


In [7]:
import pybamm

class PyBammBattery(Battery):
    def __init__(self, capacity, initial_soc=1, min_soc=0, charge_rate=100):
        super().__init__(capacity, initial_soc, min_soc, charge_rate)
        self.nominal_voltage = 120  # Nominal voltage in V
        self.capacity_wh = capacity  # Capacity in Watt-hours
        self.capacity_ah = self.capacity_wh / self.nominal_voltage  # Convert capacity from W.h to A.h
        self.model = pybamm.lithium_ion.DFN()
        self.model.options["calculate discharge energy"] = "true"
        parameter_values = self.model.default_parameter_values
        parameter_values.update({"Nominal cell capacity [A.h]": self.capacity_ah})
        self.simulation = pybamm.Simulation(self.model, parameter_values=parameter_values)
        self.simulation.set_initial_soc(initial_soc)

    def discharge(self, power_amount, duration_hours):
        """
        Discharge the battery with a given power for a specific duration.
        
        Args:
            power_amount (float): Power in Watts.
            duration (float): Duration in hours.
        """
        # Convert duration to hours for the experiment string
        
        # Define the experiment string using the power amount and duration
        experiment_string = f"Discharge at {power_amount} W for {duration_hours} hours"
        
        # Create an experiment with the string
        experiment = pybamm.Experiment([experiment_string])
        
        # Run the simulation with the experiment
        self.simulation = pybamm.Simulation(self.model, experiment=experiment, parameter_values=self.simulation.parameter_values)
        solution = self.simulation.solve()

        # Extract discharge energy and capacity
        discharge_capacity = solution["Discharge capacity [A.h]"].data[-1]
        discharge_energy = solution["Discharge energy [W.h]"].data[-1]
        print(f"Discharge capacity: {discharge_capacity:.2f} A.h")
        print(f"Discharge energy: {discharge_energy:.2f} W.h")

        return solution
    
    def charge(self, power_amount, duration_hours):
        """
        Charge the battery with a given power for a specific duration.
        
        Args:
            power_amount (float): Power in Watts.
            duration (float): Duration in seconds.
        """
        
        
        # Define the experiment string using the power amount and duration
        experiment_string = f"Charge at {-power_amount} W for {duration_hours} hours"  # Negative power for charging
        
        # Create an experiment with the string
        experiment = pybamm.Experiment([experiment_string])
        
        # Run the simulation with the experiment
        self.simulation = pybamm.Simulation(self.model, experiment=experiment, parameter_values=self.simulation.parameter_values)
        solution = self.simulation.solve()

        # Extract charge energy and capacity
        charge_energy = solution["Discharge energy [W.h]"].data[-1]  # Note: Still "Discharge energy"
        print(f"Charge energy: {charge_energy:.2f} W.h")

        return solution

battery = PyBammBattery(capacity=5000)  # 5000 W.h capacity
output = battery.discharge(power_amount=2, duration_hours=1)  # Discharge at 100 W for 1 hour
output = battery.discharge(power_amount=2, duration_hours=1)  # Discharge at 100 W for 1 hour

# battery.charge(power_amount=1, duration_hours=1)  # Charge at 50 W for 1 hour≈


Discharge capacity: 0.53 A.h
Discharge energy: 0.00 W.h
Discharge capacity: 0.53 A.h
Discharge energy: 0.00 W.h


In [23]:
import pybamm

class PyBammBattery(Battery):
    def __init__(self, capacity, initial_soc=1, min_soc=0, charge_rate=100):
        super().__init__(capacity, initial_soc, min_soc, charge_rate)
        self.nominal_voltage = 5  # Nominal voltage in V
        self.capacity_wh = capacity  # Capacity in Watt-hours
        self.capacity_ah = self.capacity_wh / self.nominal_voltage  # Convert capacity from W.h to A.h
        self.model = pybamm.lithium_ion.DFN()
        self.model.options["calculate discharge energy"] = "true"
        parameter_values = self.model.default_parameter_values
        parameter_values.update({"Nominal cell capacity [A.h]": self.capacity_ah})
        # usually the nominal cell capacity is like 5.0 A.h
        parameter_values.update({
            "Nominal cell capacity [A.h]": self.capacity_ah,
            "Lower voltage cut-off [V]": 0,
            "Upper voltage cut-off [V]": 10
        })
        print("nominal capacity is: " + str(self.capacity_ah))
        self.simulation = pybamm.Simulation(self.model, parameter_values=parameter_values)
        self.solution = self.simulation.solve(t_eval=[0, 1])  # Initial solve with a short time interval
        self.min_soc = min_soc
        self.current_soc = initial_soc

    def discharge(self, power_amount, duration_hours):
        """
        Discharge the battery with a given power for a specific duration.
        
        Args:
            power_amount (float): Power in Watts.
            duration (float): Duration in hours.
        """
        # Convert duration to hours for the experiment string
        
        # Define the experiment string using the power amount and duration
        experiment_string = f"Discharge at {power_amount} W for {duration_hours} hours"
        
        # Create an experiment with the string
        experiment = pybamm.Experiment([experiment_string])
        
        # Run the simulation with the experiment, using the previous solution as the initial state
        self.simulation = pybamm.Simulation(self.model, experiment=experiment, parameter_values=self.simulation.parameter_values)
        self.solution = self.simulation.solve(initial_soc=self.current_soc)

        # Extract discharge energy and capacity
        discharge_capacity = self.solution["Discharge capacity [A.h]"].data[-1]
        print(f"Discharge capacity: {discharge_capacity:.2f} A.h")
        # Update SOC and charge level
        self.current_soc -= discharge_capacity / self.capacity_ah
        self.current_soc = max(self.current_soc, self.min_soc)
        print(f"current_soc: {self.current_soc} ")
        

        return self.solution
    
    def charge(self, power_amount, duration_hours):
        """
        Charge the battery with a given power for a specific duration.
        
        Args:
            power_amount (float): Power in Watts.
            duration (float): Duration in seconds.
        """
        
        
        # Define the experiment string using the power amount and duration
        experiment_string = f"Charge at {power_amount} W for {duration_hours} hours"  # Negative power for charging
        
        # Create an experiment with the string
        experiment = pybamm.Experiment([experiment_string])
        
        # Run the simulation with the experiment, using the previous solution as the initial state
        self.simulation = pybamm.Simulation(self.model, experiment=experiment, parameter_values=self.simulation.parameter_values)
        self.solution = self.simulation.solve(initial_soc=self.current_soc)

        # Extract charge energy and capacity
        charge_capacity = (-1)*self.solution["Discharge capacity [A.h]"].data[-1]
        print(f"Charge capacity: {charge_capacity:.2f} A.h")
        
        self.current_soc += charge_capacity / self.capacity_ah
        self.current_soc = min(1, self.current_soc)
        print(f"current_soc: {self.current_soc} ")
        

        return self.solution

battery = PyBammBattery(capacity=25)  # 5000 W.h capacity
output = battery.discharge(power_amount=2, duration_hours=1)  # Discharge at 100 W for 1 hour
output = battery.discharge(power_amount=2, duration_hours=1)  # Discharge at 100 W for 1 hour
output = battery.discharge(power_amount=2, duration_hours=1)  # Discharge at 100 W for 1 hour
output = battery.charge(power_amount=1, duration_hours=1)  # Discharge at 100 W for 1 hour
# output = battery.charge(power_amount=1, duration_hours=1)  # Discharge at 100 W for 1 hour
# output = battery.charge(power_amount=2, duration_hours=1)  # Discharge at 100 W for 1 hour



nominal capacity is: 5.0
Discharge capacity: 0.53 A.h
current_soc: 0.8930208251975268 
Discharge capacity: 0.54 A.h
current_soc: 0.7845876111203313 
Discharge capacity: 0.55 A.h
current_soc: 0.6751512101442897 
Charge capacity: 0.26 A.h
current_soc: 0.726199836189548 


In [ ]:
class PyBammBattery(Battery):
    def __init__(self, capacity, initial_soc=1, min_soc=0, charge_rate=100):
        super().__init__(capacity, initial_soc, min_soc, charge_rate)
        self.nominal_voltage = 120  # Nominal voltage in V
        self.capacity_wh = capacity   # Convert capacity from W.s to W.h
        self.capacity_ah = self.capacity_wh / self.nominal_voltage  # Convert capacity from W.h to A.h
        self.model = pybamm.lithium_ion.DFN()
        self.model.options["calculate discharge energy"] = "true"
        parameter_values = self.model.default_parameter_values
        parameter_values.update({"Nominal cell capacity [A.h]": self.capacity_ah})
        self.simulation = pybamm.Simulation(self.model, parameter_values=parameter_values)
        self.simulation.set_initial_soc(initial_soc)
        self.simulation.solve([0, 1])  # Initialize the simulation
        self.current_soc = initial_soc

        

    def discharge(self, power_amount, time_step):  # the power_amount here is a positive value
        energy_amount_wh = power_amount * time_step # unit is watt, time_step unit is hour

        # Calculate the current needed to discharge the given energy
        current_a = power_amount / self.nominal_voltage

        self.simulation.model.variables["Current [A]"] = -current_a  # Set discharge current (negative)
        self.simulation.step(dt=time_step)
        
        amount = self.simulation.solution["Discharge capacity [A.h]"].data[-1]
        # print("discharge amount is: " + str(amount))
        self.current_soc = self.current_soc - self.simulation.solution["Discharge capacity [A.h]"].data[-1] / self.capacity_ah
        # print("capacity_ah is: " + str(capacity_ah))
        print("discharge capacity is: " + str(self.simulation.solution["Discharge capacity [A.h]"].data[-1]))
        print("discharge energy is: " + str(self.simulation.solution["Discharge capacity [A.h]"].data[-1] * self.nominal_voltage))
        self.charge_level = self.capacity_wh * self.current_soc
        if self.current_soc  < self.min_soc:
            self.current_soc = self.min_soc
            self.charge_level = self.capacity_wh * self.current_soc
            return 0 # Limit discharge to minimum SOC
        
        
        return energy_amount_wh * 3600  # Return the energy discharged in W.s

    def charge(self, power_amount, time_step):
        energy_amount_wh = power_amount * time_step # unit is watt, time_step unit is hour

        # Calculate the current needed to discharge the given energy
        current_a = power_amount / self.nominal_voltage

        self.simulation.model.variables["Current [A]"] = current_a  # Set charge current (positive)
        self.simulation.step(dt=time_step)
        
        
        self.current_soc = self.current_soc  + self.simulation.solution["Discharge capacity [A.h]"].data[-1] / self.capacity_ah
        # print("capacity_ah is: " + str(capacity_ah))
        print("discharge capacity is: " + str(self.simulation.solution["Discharge capacity [A.h]"].data[-1]))
        print("discharge energy is: " + str(self.simulation.solution["Discharge capacity [A.h]"].data[-1] * self.nominal_voltage))
        
        self.charge_level = self.capacity_wh * self.current_soc
        if self.current_soc  > 1:
            self.current_soc = 1
            self.charge_level = self.capacity_wh * self.current_soc
            return 0 # Limit discharge to minimum SOC
    

        return energy_amount_wh * 3600  # Return the energy charged in W.s

In [ ]:
newBattery = PyBammBattery(capacity=3600000)
print("current soc: " + str(newBattery.current_soc))
# newBattery.discharge(1000000,1)
print("current soc: " + str(newBattery.current_soc))

newBattery.charge(50000,1)
print("current soc: " + str(newBattery.current_soc))


In [ ]:
import pybamm

class PyBammBattery(Battery):
    def __init__(self, capacity, initial_soc=1, min_soc=0, charge_rate=100):
        super().__init__(capacity, initial_soc, min_soc, charge_rate)
        self.nominal_voltage = 120  # Nominal voltage in V
        self.capacity_wh = capacity  # Capacity in Watt-hours
        self.capacity_ah = self.capacity_wh / self.nominal_voltage  # Convert capacity from W.h to A.h
        self.model = pybamm.lithium_ion.DFN()
        self.model.options["calculate discharge energy"] = "true"
        
        # Update the parameter values, including nominal voltage
        parameter_values = self.model.default_parameter_values
        parameter_values.update({
            "Nominal cell capacity [A.h]": self.capacity_ah,
            "Nominal cell voltage [V]": self.nominal_voltage
        })
        
        # Initialize the simulation with the given parameters and initial SOC
        self.simulation = pybamm.Simulation(self.model, parameter_values=parameter_values)
        self.solution = self.simulation.solve([0, 1], initial_soc=initial_soc)
        self.current_soc = initial_soc

    def discharge(self, power_amount, time_step):
        """
        Discharge the battery with a given power for a specific duration.
        
        Args:
            power_amount (float): Power in Watts.
            time_step (float): Duration in hours.
        """
        # Define the experiment string for discharge
        experiment_string = f"Discharge at {power_amount} W for {time_step} hours"
        experiment = pybamm.Experiment([experiment_string])
        
        # Run the simulation with the experiment, using the previous solution as the initial state
        self.simulation = pybamm.Simulation(self.model, experiment=experiment, parameter_values=self.simulation.parameter_values)
        self.solution = self.simulation.solve(initial_soc=self.current_soc)
        
        # Update SOC and charge level
        discharge_capacity = self.solution["Discharge capacity [A.h]"].data[-1]
        discharge_energy = self.solution["Discharge energy [W.h]"].data[-1]
        self.current_soc -= discharge_capacity / self.capacity_ah
        self.current_soc = max(self.current_soc, self.min_soc)  # Ensure SOC doesn't drop below min
        self.charge_level = self.capacity_wh * self.current_soc
        
        print(f"Discharge capacity: {discharge_capacity:.2f} A.h")
        print(f"Discharge energy: {discharge_energy:.2f} W.h")
        
        return discharge_energy * 3600  # Return energy in Joules

    def charge(self, power_amount, time_step):
        """
        Charge the battery with a given power for a specific duration.
        
        Args:
            power_amount (float): Power in Watts.
            time_step (float): Duration in hours.
        """
        # Define the experiment string for charge
        experiment_string = f"Charge at {-power_amount} W for {time_step} hours"  # Negative power for charging
        experiment = pybamm.Experiment([experiment_string])
        
        # Run the simulation with the experiment, using the previous solution as the initial state
        self.simulation = pybamm.Simulation(self.model, experiment=experiment, parameter_values=self.simulation.parameter_values)
        self.solution = self.simulation.solve(initial_soc=self.current_soc)
        
        # Update SOC and charge level
        charge_capacity = self.solution["Discharge capacity [A.h]"].data[-1]  # It should be "Charge capacity" ideally
        charge_energy = self.solution["Discharge energy [W.h]"].data[-1]
        self.current_soc += charge_capacity / self.capacity_ah
        self.current_soc = min(self.current_soc, 1.0)  # Ensure SOC doesn't exceed 100%
        self.charge_level = self.capacity_wh * self.current_soc
        
        print(f"Charge capacity: {charge_capacity:.2f} A.h")
        print(f"Charge energy: {charge_energy:.2f} W.h")
        
        return charge_energy * 3600  # Return energy in Joules

battery = PyBammBattery(capacity=5000)  # 5000 W.h capacity
output = battery.discharge(power_amount=2, time_step=1)  # Discharge at 2 W for 1 hour
output = battery.discharge(power_amount=2, time_step=1)  # Discharge again at 2 W for 1 hour
battery.charge(power_amount=2, time_step=1)  # Charge at 2 W for 1 hour
